In [1]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, inspect, select

---
## Conexión a BigQuery

**NOTA:** Antes de conectar con BigQuery, es necesario designar las credenciales dentro de tu ambiente.

__en Windows:__

1. Corre el siguiente command en `cmd.exe`, con privilegios de administrador Windows:

 - `setx GOOGLE_APPLICATION_CREDENTIALS "C:\{ruta}\bq_keys.json" /M`

    (reemplace `{ruta}` con la ruta de `bq_keys.json` en tu ambiente)

2. Confirme la configuración con:

 - `echo %GOOGLE_APPLICATION_CREDENTIALS%`

---
### Establecer conexión

In [2]:
# Nombres de proyecto y dataset en Google Cloud
project_id = 'coastal-height-421718'
dataset = 'Yelp_Google'

# Crear motor de conexión
db_uri = f'bigquery://{project_id}/{dataset}'
engine = create_engine(db_uri)

### Prueba de conexión

In [3]:
# Instanciar objeto `MetaData`
metadata = MetaData()
# Cargar las tablas del dataset
metadata.reflect(bind=engine)
# Imprimir nombre de todo tabla
for nom_tabla in metadata.tables:
    print(nom_tabla)

/home/henry/PF/env-pf/lib/python3.10/site-packages/google/cloud/bigquery/client.py:585: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


y_business
y_business_attrib
y_checkin
y_review
y_tip
y_user


---
### Ejemplo query: Reviews de Darden
En este ejemplo, cargamos todos los reviews correpondientes a Darden.

1. Cargamos un dict con los nombres de restaurantes de Darden y su competidores.

In [4]:
# Importar archivo `carga_data.py`
import utils.carga_data as cd

# Diccionario de compañias con sus repectivas listas de marcas 
brands = cd.cargar_brands()
brands.keys()

dict_keys(['darden', 'bloomin', 'brinker', 'texasroadhouse'])

In [5]:
brands['darden']

['Olive Garden Italian Restaurant',
 'LongHorn Steakhouse',
 "Cheddar's Scratch Kitchen",
 'Yard House',
 'The Capital Grille',
 'Seasons 52',
 'Bahama Breeze',
 "Eddie V's",
 "Ruth's Chris Steak House"]

2. Imprimimos las columnas de tabla `y_review`

In [6]:
# Instanciar objeto `inspect`
inspector = inspect(engine)
# Identificar la tabla de interés
nombre_tabla = 'y_review'

# Colectar e imprimir los nombre de columnas de `y_review`
col = inspector.get_columns(nombre_tabla)
for c in col:
    print(c['name'])

name
user_id
business_id
stars
useful
funny
cool
text
date
review_id


3. Ejecutar query

In [ ]:
# Definir context metadata
metadata = MetaData()

y_review = Table('y_review', metadata,
                   Column('name', String),
                   Column('text', String)
                   )
# Contruir un SQL SELECT, con condicional de `name` se encuentre en in la lista `brands['darden']`
stmt = select(y_review.c.name, y_review.c.text).where(y_review.c.name.in_(brands['darden']))

# Execute the query and fetch the results
with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()

# Imprimir cada fila del resultado
for row in results:
    print(row)

4. Gestionar datos localmente

In [8]:
import pandas as pd

df_review_darden = pd.DataFrame(results)
df_review_darden.sample(3)

,name,text
402,Seasons 52,Bottom line is that you get really good food a...
2718,LongHorn Steakhouse,This restaurant sucks the waitress was slow an...
1403,Bahama Breeze,The food and service was good. Enjoyed the wa...
